# Demostración de Threading con Estado Compartido

Este notebook demuestra cómo ejecutar dos hilos simultáneamente y manejar adecuadamente el estado de una variable compartida usando sincronización.

In [1]:
import threading
import time
import random
from datetime import datetime

# Variable compartida y lock para sincronización
shared_counter = 0
counter_lock = threading.Lock()
execution_log = []
log_lock = threading.Lock()

def thread_function_1(name, iterations=5):
    """Función que incrementa el contador compartido"""
    global shared_counter
    
    for i in range(iterations):
        # Simular trabajo
        time.sleep(random.uniform(0.1, 0.3))
        
        # Acceso seguro a la variable compartida
        with counter_lock:
            old_value = shared_counter
            shared_counter += 1
            new_value = shared_counter
        
        # Log seguro
        with log_lock:
            execution_log.append({
                'thread': name,
                'iteration': i + 1,
                'operation': 'INCREMENT',
                'old_value': old_value,
                'new_value': new_value,
                'timestamp': datetime.now().strftime('%H:%M:%S.%f')[:-3]
            })
        
        print(f"{name} - Iteración {i+1}: {old_value} -> {new_value}")

def thread_function_2(name, iterations=5):
    """Función que decrementa el contador compartido"""
    global shared_counter
    
    for i in range(iterations):
        # Simular trabajo
        time.sleep(random.uniform(0.1, 0.3))
        
        # Acceso seguro a la variable compartida
        with counter_lock:
            old_value = shared_counter
            shared_counter -= 1
            new_value = shared_counter
        
        # Log seguro
        with log_lock:
            execution_log.append({
                'thread': name,
                'iteration': i + 1,
                'operation': 'DECREMENT',
                'old_value': old_value,
                'new_value': new_value,
                'timestamp': datetime.now().strftime('%H:%M:%S.%f')[:-3]
            })
        
        print(f"{name} - Iteración {i+1}: {old_value} -> {new_value}")

# Resetear variables para nueva ejecución
shared_counter = 0
execution_log = []

print("=== INICIANDO DEMOSTRACIÓN DE THREADING ===")
print(f"Valor inicial del contador: {shared_counter}")
print("\n--- Creando y ejecutando hilos ---")

# Crear los hilos
thread1 = threading.Thread(target=thread_function_1, args=("HILO-INCREMENTO", 3))
thread2 = threading.Thread(target=thread_function_2, args=("HILO-DECREMENTO", 3))

# Iniciar ambos hilos simultáneamente
start_time = time.time()
thread1.start()
thread2.start()

# Esperar a que ambos hilos terminen
thread1.join()
thread2.join()
end_time = time.time()

print("\n=== RESULTADOS ===")
print(f"Valor final del contador: {shared_counter}")
print(f"Tiempo total de ejecución: {end_time - start_time:.2f} segundos")
print(f"Total de operaciones realizadas: {len(execution_log)}")

print("\n--- Log detallado de operaciones ---")
for entry in execution_log:
    print(f"[{entry['timestamp']}] {entry['thread']} - {entry['operation']}: {entry['old_value']} -> {entry['new_value']}")

print("\n=== VERIFICACIÓN DE THREAD SAFETY ===")
# Verificar que no hay condiciones de carrera
expected_final_value = 0  # 3 incrementos - 3 decrementos = 0
print(f"Valor esperado: {expected_final_value}")
print(f"Valor obtenido: {shared_counter}")
print(f"Thread safety mantenido: {shared_counter == expected_final_value}")

# Mostrar estadísticas por hilo
increment_ops = [entry for entry in execution_log if entry['operation'] == 'INCREMENT']
decrement_ops = [entry for entry in execution_log if entry['operation'] == 'DECREMENT']

print(f"\nOperaciones de incremento: {len(increment_ops)}")
print(f"Operaciones de decremento: {len(decrement_ops)}")
print("\n¡Demostración completada exitosamente!")

=== INICIANDO DEMOSTRACIÓN DE THREADING ===
Valor inicial del contador: 0

--- Creando y ejecutando hilos ---
HILO-INCREMENTO - Iteración 1: 0 -> 1
HILO-DECREMENTO - Iteración 1: 1 -> 0HILO-DECREMENTO - Iteración 1: 1 -> 0

HILO-DECREMENTO - Iteración 2: 0 -> -1
HILO-DECREMENTO - Iteración 2: 0 -> -1
HILO-INCREMENTO - Iteración 2: -1 -> 0
HILO-DECREMENTO - Iteración 3: 0 -> -1
HILO-INCREMENTO - Iteración 2: -1 -> 0
HILO-DECREMENTO - Iteración 3: 0 -> -1
HILO-INCREMENTO - Iteración 3: -1 -> 0

=== RESULTADOS ===
Valor final del contador: 0
Tiempo total de ejecución: 0.65 segundos
Total de operaciones realizadas: 6

--- Log detallado de operaciones ---
[16:43:10.718] HILO-INCREMENTO - INCREMENT: 0 -> 1
[16:43:10.759] HILO-DECREMENTO - DECREMENT: 1 -> 0
[16:43:10.915] HILO-DECREMENTO - DECREMENT: 0 -> -1
[16:43:11.016] HILO-INCREMENTO - INCREMENT: -1 -> 0
[16:43:11.098] HILO-DECREMENTO - DECREMENT: 0 -> -1
[16:43:11.238] HILO-INCREMENTO - INCREMENT: -1 -> 0

=== VERIFICACIÓN DE THREAD SAF